In [1]:
%matplotlib inline
import os
import sys
sys.path.append('..')
import shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pyemu
import glob
import zipfile
# flag to decide if run everything or just visualize results
run_all_flag = False

In [2]:
import freyberg_setup
working_dir = freyberg_setup.WORKING_DIR_KR
pst_name = freyberg_setup.PST_NAME_KR
freyberg_setup.setup_pest_kr()


if not run_all_flag:
    shutil.copy2('sweep.zip',os.path.join(working_dir,'sweep.zip'))
    with zipfile.ZipFile(os.path.join(working_dir,'sweep.zip')) as inzip:
        inzip.extractall(working_dir)


changing model workspace...
   freyberg_kr
FloPy is using the following  executable to run the model: C:\Users\FienenMichaelN\source\repos\GW1876\activities\freyberg_k_and_r_response_surface\freyberg_kr\mfnwt.exe

                                  MODFLOW-NWT-SWR1 
    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUNDWATER-FLOW MODEL
                             WITH NEWTON FORMULATION
                             Version 1.0.9 07/01/2014                        
                    BASED ON MODFLOW-2005 Version 1.11.0 08/08/2013                       

                    SWR1 Version 1.03.0 08/30/2013                       

 Using NAME file: freyberg.nam 
 Run start date and time (yyyy/mm/dd hh:mm:ss): 2018/10/08 13:02:34

 Solving:  Stress period:     1    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     2    Time step:     1    Groundwater-Flow Eqn.
 Solving:  Stress period:     3    Time step:     1    Groundwater-Flow Eqn.
 Run end date and time (yyy

C:\Users\FienenMichaelN\source\repos\pyemu\pyemu\utils\gw_utils.py:1747: PyemuWarning: Flow out aggregation for segment has changed. Now returning flow out at bottom of seg ...
  "Now returning flow out at bottom of seg ...", PyemuWarning)
C:\Users\FienenMichaelN\source\repos\pyemu\pyemu\utils\gw_utils.py:1607: PyemuWarning: Flow out (flout) of grouped segments will be aggregated... 
  warnings.warn("Flow out (flout) of grouped segments will be aggregated... ", PyemuWarning)


writing 'sfr_obs.config' to sfr_obs.config
                                    obsnme     obsval  weight    obgnme
c001fr05c04_19791231  c001fr05c04_19791231  29.453472     0.0  forehead
c001fr16c17_19791231  c001fr16c17_19791231  21.617983     0.0  forehead
noptmax: 0
noptmax: 8


## First read in the PST file and find what are the starting values for K and R

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.parameter_data

In [ ]:
inpst.parameter_data.loc['rch_1','partrans']= 'fixed'
inpst.parameter_data.loc['rch_0','partrans']= 'log'
inpst.parameter_data.loc['rch_0','parval1']= 8.0E-1
inpst.parameter_data.loc['hk', 'parval1'] = 12.0
inpst.observation_data.loc[inpst.observation_data.obgnme=='calflux','weight']=0.0
inpst.control_data.noptmax=0
inpst.write(os.path.join(working_dir,pst_name))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name))
os.chdir('..')

In [ ]:
inpst.phi_components

## We can import a function to explore a range of K and R values to make a response surface

In [ ]:
import response_surface as resurf

## First we should write out the control file with new bounds to bracket where values will be explored

## `rch_0` already spans the bounds we want, but `hk` is too wide

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
par = inpst.parameter_data
inpst.parameter_data.loc['hk', 'parval1'] = 12.0
par.loc['hk', 'parlbnd'] = 3
par.loc['hk', 'parubnd'] = 12

inpst.write(os.path.join(working_dir,pst_name))

In [ ]:
inpst.parameter_data

In [ ]:
inpst.observation_data.loc[inpst.nnz_obs_names]

## Run the `run_respsurf` function to calculate the response surface

In [ ]:
if run_all_flag:
    resurf.run_respsurf(par_names=['hk','rch_0'], pstfile=pst_name)

In [ ]:
resurf.plot_response_surface(parnames=['hk','rch_0'], pstfile=pst_name);

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))
inpst.parameter_data.loc['hk', 'parval1'] = 12.0
inpst.parameter_data.loc['rch_0', 'parval1'] = 0.8
inpst.parameter_data.loc['rch_0', 'partrans'] = 'log'
inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name))
os.chdir('..')

### Now let's run PEST++ and see what we get for an estimate of the parameters
 * set NOPTMAX=-1
 * run PEST++

In [ ]:
pars = pd.read_csv(os.path.join(working_dir,pst_name.replace('.pst','.ipar')))
pars

In [ ]:
sc = pyemu.Schur(os.path.join(working_dir,pst_name.replace('.pst','.jcb')))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

# What if we also have a calibration flux observation?

We can set this up by simply assigning non-zero weight to the observation in the group called `calflux`

We'll read in the PST file, change just that one value and write out a new PST file.

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name))

In [ ]:
inpst.parameter_data

In [ ]:
inpst.parameter_data.loc['rch_1','partrans']= 'fixed'
###
# Let's fix porosity to avoid getting NaN in our correlation matrices

inpst.parameter_data.loc['porosity','partrans']= 'fixed'

inpst.parameter_data.loc['hk','parval1']= 12.0

inpst.observation_data.loc[inpst.observation_data.obgnme=='calflux','weight']=10
inpst.control_data.noptmax=0
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r1.pst')))

In [ ]:
inpst.parameter_data

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r1.pst')))
os.chdir('..')

In [ ]:
inpst.phi_components

In [ ]:
if run_all_flag:
    resurf.run_respsurf(par_names=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r1.pst'))

In [ ]:
resurf.plot_response_surface(parnames=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r1.pst'));

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name.replace('.pst','.r1.pst')))
inpst.parameter_data.loc['hk', 'parval1'] = 12
inpst.parameter_data.loc['rch_0', 'parval1'] = 0.8

inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r1.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r1.pst')))
os.chdir('..')

### We can look at the correlation again

In [ ]:
pars = pd.read_csv(os.path.join(working_dir,pst_name.replace('.pst','.r1.ipar')))
pars

In [ ]:
sc = pyemu.Schur(os.path.join(working_dir,pst_name.replace('.pst','.r1.jcb')))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

In [ ]:
sc.jco.to_dataframe().loc[inpst.nnz_obs_names]

## OK - that was bonkers. Maybe a little less weight?

In [ ]:
inpst.parameter_data.loc['rch_1','partrans']= 'fixed'

inpst.observation_data.loc[inpst.observation_data.obgnme=='calflux','weight']=0.001
inpst.control_data.noptmax=0
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r2.pst')))

In [ ]:
inpst.parameter_data


In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r2.pst')))
os.chdir('..')

In [ ]:
inpst.phi_components

In [ ]:
if run_all_flag:
    resurf.run_respsurf(par_names=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r2.pst'))

In [ ]:
resurf.plot_response_surface(parnames=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r2.pst'));

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name.replace('.pst','.r2.pst')))
inpst.parameter_data.loc['hk', 'parval1'] = 12
inpst.parameter_data.loc['rch_0', 'parval1'] = 0.8

inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r2.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r2.pst')))
os.chdir('..')

## Let's check out correlation again

In [ ]:
pars = pd.read_csv(os.path.join(working_dir,pst_name.replace('.pst','.r2.ipar')))
pars

In [ ]:
sc = pyemu.Schur(os.path.join(working_dir,pst_name.replace('.pst','.r2.jcb')))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()

## Can we find a happy medium? Perhaps if we balance the components of the objective function

In [ ]:
inpst.parameter_data.loc['rch_1','partrans']= 'fixed'
inpst.observation_data.loc[inpst.observation_data.obgnme=='calflux','weight']=0.05
inpst.control_data.noptmax=0
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r3.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r3.pst')))
os.chdir('..')

In [ ]:
inpst.phi_components

In [ ]:
if run_all_flag:
    resurf.run_respsurf(par_names=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r3.pst'))

In [ ]:
resurf.plot_response_surface(parnames=['hk','rch_0'], pstfile=pst_name.replace('.pst','.r3.pst'));

In [ ]:
inpst = pyemu.Pst(os.path.join(working_dir,pst_name.replace('.pst','.r3.pst')))
inpst.parameter_data.loc['hk', 'parval1'] = 7
inpst.parameter_data.loc['rch_0', 'parval1'] = 1.0
inpst.parameter_data.loc['rch_0', 'parlbnd'] = 0.6


inpst.control_data.noptmax =  -1
inpst.write(os.path.join(working_dir,pst_name.replace('.pst','.r3.pst')))

In [ ]:
os.chdir(working_dir)
pyemu.os_utils.run('pestpp {0}'.format(pst_name.replace('.pst','.r3.pst')))
os.chdir('..')

In [ ]:
pars = pd.read_csv(os.path.join(working_dir,pst_name.replace('.pst','.r3.ipar')))
pars

In [ ]:
sc = pyemu.Schur(os.path.join(working_dir,pst_name.replace('.pst','.r3.jcb')))
covar = pyemu.Cov(sc.xtqx.x, names=sc.xtqx.row_names)
covar.df()

In [ ]:
R = covar.to_pearson()
R.df()